In [37]:
from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt

import random
import time

import attr
from attr.validators import instance_of

from pathlib import Path
import os

from typing import List, Dict, NewType, Callable, Any, Iterable, SupportsIndex

In [45]:
T_BenchName = NewType("bench_name", str)
T_dt = NewType("dt", float)
T_dts = NewType("dts", List[T_dt])


@attr.s(frozen=True)
class BenchDescription:
    name: T_BenchName = attr.ib(validator=instance_of(str))
    n_execs: int = attr.ib(validator=instance_of(int))
    total_time: float = attr.ib(validator=instance_of(float | None))
    mean: float = attr.ib(validator=instance_of(float | None))
    median: float = attr.ib(validator=instance_of(float | None))
    std: float = attr.ib(validator=instance_of(float | None))


@attr.s(frozen=True)
class BenchData:
    """
    Attributes:
    -----------
    - `name (str):` Nombre del benchmark.
    - `dts (List[dt]):` Guarda el `dt` de cada ejecución.
    - `n_execs (int):`  len(dts) ~~ Número de ejecuciones.
    - `is_empty (bool):` n_execs==0 -> True.
    - `total_time (float):` sum(dts) ~~ Suma de todos los tiempos de ejecución.
    - `mean (float):`   np.mean(dts) ~~ Media.
    - `median (float):` np.median(dts) ~~ Mediana.
    - `std (float):` np.std(dts) ~~ Desv. Standard.
    - `description (BenchmarkDesc)`: Retorna un diccionario con los valores representativos.

    Methods:
    --------
    """
    name: T_BenchName = attr.ib(validator=instance_of(str))
    dts: T_dts = attr.ib(validator=instance_of(list), factory=list, repr=False)

    def __len__(self) -> int:
        return len(self.dts)
    
    @property
    def n_execs(self) -> int:
        return len(self)
    
    @property
    def is_empty(self) -> bool: return self.n_execs == 0
    @property
    def not_empty(self) -> bool: return not self.is_empty

    @property
    def total_time(self) -> float | None:
        if self.not_empty: return np.sum(self.dts)
    
    @property
    def mean(self) -> float | None:
        if self.not_empty: return np.mean(self.dts)
    
    @property
    def median(self) -> float | None:
        if self.not_empty: return np.median(self.dts)
    
    @property
    def std(self) -> float | None:
        if self.not_empty: return np.std(self.dts)

    @property
    def description(self) -> BenchDescription:
        return BenchDescription(
            name = self.name,
            n_execs = self.n_execs,
            total_time = self.total_time,
            mean = self.mean,
            median = self.median,
            std = self.std
        )

    def append(self, dt: T_dt) -> None:
        self.dts.append(dt)

    def reset(self) -> None:
        """ Elimina todos los `dts` guardados."""
        self.dts.clear()
    
    def plot(self, log=False, *args, **kwargs):
        plt.hist(self.dts, log=log, *args, **kwargs)
        plt.show()


class Benchmarks(Dict[T_BenchName, BenchData]):
    """ Benchmark de todas las funciones decoradas con `dt`."""
    def __init__(self, bench_names: Iterable[str]):
        assert len(bench_names) == len(set(bench_names)), "No puede haber claves repetidas."
        super().__init__({name: BenchData(name=name) for name in bench_names})
        
        # Va a guardar el nombre del bench que está calculando.
        # Para evitar el problema de los benchs anidados. Que una función llama a otra
        # Y la suma de los tiempos no representa el tiempo total.
        # Ver si tiene sentido implementarlo. O mejor un Warning, pero relentizaría.
        # self.name_calculating: str | None = None

    @property
    def time_now(self) -> float:
        return time.perf_counter()

    def dt(self, name: T_BenchName) -> Callable[..., Any]:
        """
        Decorador: Calcula el tiempo que tarda en ejecutar la función decorada.
        - name (str): Key donde se almacena 
        - Se almacena dentro de `Benchmark` con key `name` en una lista.
        """
        def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
            def wrapper(*args, **kwargs):
                t_i = self.time_now
                # Codigo antes de ejecutar func
                result = func(*args, **kwargs)
                # Codigo antes de despues func
                dt = self.time_now - t_i
                self.add_dt(name, dt)
                return result
            return wrapper
        return decorator

    def add_dt(self, bench_name: T_BenchName, dt: T_dt) -> None:
        """ Appendea `dt` dentro de `bench_name`."""
        self[bench_name].append(dt)
    
    def description(self, bench_name: T_BenchName) -> BenchDescription:
        """ Retorna la descripción de un benchmark en concreto."""
        return self[bench_name].description

    def descriptions(self) -> Dict[T_BenchName, BenchDescription]:
        """ Retorna la descripción de todos los benchmarks."""
        return {bench_name: self.description(bench_name) for bench_name in self.keys()}

    def reset(self, bench_name: T_BenchName = None) -> None:
        """ Resetea todos los `dts` si no se especifica uno en particular."""
        if bench_name is None:
            for bench_data in self.values():
                bench_data.reset()
        else:
            assert isinstance(bench_name, str)
            self[bench_name].reset()

bench = Benchmarks(("f_1", "f_2"))

@bench.dt(name="f_1")
def func_1():
    return random.random()*1000

@bench.dt(name="f_2")
def func_2():
    return np.log(func_1())

for _ in range(100):
    func_1()
    func_2()

# FIXME: Por alguna razón la primera vez que se accede al wrapper tarda levemente mas.
# Capás es alguna tool de python, que hace algo la primera vez que agrega un elemento a una lista, o algo x el estilo.